In [ ]:
import numpy as np                                    #arrays and matrix math
import pandas as pd                                   #work with DataFrames
import matplotlib.pyplot as plt                       #plotting and visualization
import matplotlib.dates as mdates                     #datetime formate in plots

import keras.backend as K                             #keras backend
import tensorflow as tf                               #tensor operations
import h5py                                           #import h5 files
import os                                             #OS operations
import time                                           #timing and clock time
import scipy.signal as signal                         #signal processing
from scipy.io import loadmat                          #load MatLab m-files
from scipy.stats.qmc import LatinHypercube

from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer
from skimage.metrics import mean_squared_error as image_mse         #Mean Squared Error
from skimage.metrics import structural_similarity as image_ssim     #Structural Similarity Index
from skimage.metrics import peak_signal_noise_ratio as image_psnr   #Peak Signal-Noise Ratio


In [ ]:
# Function to open H5 file as Pandas DataFrame
def open_fiber_H5_2_arr(folder_n=1, file_n=0, xstart=4950, xend=5150):
    if folder_n==1:
        fold = 'E:/Lytt Fiber Optics/Sintef3mH5'
    elif folder_n==2:
        fold = 'E:/Lytt Fiber Optics/Sintef10mH5'
    elif folder_n==45:
        fold = 'E:/Lytt Fiber Optics/45'
    elif folder_n==48:
        fold = 'E:/Lytt Fiber Optics/48'
    elif folder_n==109:
        fold = 'E:/Lytt Fiber Optics/109'
    elif folder_n==128:
        fold = 'E:/Lytt Fiber Optics/128'
    file = os.listdir(fold)
    file_path = os.path.join(fold, file[file_n])
    f = h5py.File(file_path, 'r')
    df = pd.DataFrame((f['Acquisition']['Raw[0]']['RawData'])).iloc[:, xstart:xend].T
    f.close
    return df

In [ ]:
# folder54, files54 = 'E:/Lytt Fiber Optics/Sintef10mH5', os.listdir(folder54)               # done
# folder64, files64 = 'E:/Lytt Fiber Optics/Sintef3mH5', os.listdir(folder64)                ########## re-do this one! (accidentally deleted)
# folder45, files45 = 'E:/Lytt Fiber Optics/45', os.listdir(folder45)                        # done
# folder48, files48 = 'E:/Lytt Fiber Optics/48', os.listdir(folder48)                        # done
# folder109, files109 = 'E:/Lytt Fiber Optics/109', os.listdir(folder109)                    # done
# folder128, files128 = 'E:/Lytt Fiber Optics/128', os.listdir(folder128)                    # done

# change folder_i, files_i to desired experiment
folder128 = 'E:/Lytt Fiber Optics/128'
print('Experiment 128 (file128) # of files: {}'.format(len(os.listdir(folder128))))

data_exp128 = pd.DataFrame(())
for i in range(len(os.listdir(folder128))):
    new_data1 = open_fiber_H5_2_arr(folder_n=128, file_n=i, xstart=4950, xend=5150)
    data_exp128 = pd.concat([data_exp128, new_data1], ignore_index=True, axis=1)
    data_exp128.to_pickle('E:/Lytt Fiber Optics/data_exp128.pkl')

In [ ]:
pd.read_csv('E:/Lytt Fiber Optics/DTS 45/20190620_142044328000_Sintef_2019_DTS.csv').iloc[:,1]

In [ ]:
folder = 'E:/Lytt Fiber Optics/DTS 109'
files = os.listdir(folder)
dts_exp109 = pd.DataFrame()
for i in range(len(files)):
    df = pd.read_csv(os.path.join(folder,files[i])).iloc[:,1]
    dts_exp109 = pd.concat([dts_exp109, df], ignore_index=True, axis=1)

In [ ]:
# def read_save_dts_data(folder, save_name, size=200):
#     print('This is Experiment {}. Saved.'.format(folder[-2:]))
#     ### folder1 == 'E:/Research/Lytt Fiber Optics/DTS Experiment 54' ###
#     ### folder2 == 'E:/Research/Lytt Fiber Optics/DTS Experiment 64' ###
#     # save depth datums
#     dts_depths = pd.read_csv(os.path.join(folder, os.listdir(folder)[0]), usecols=[1]).squeeze()[-size:]
#     dts_depths.to_pickle('dts_depthstamps.pkl')
#     # save timestamps
#     dts_timestamps = pd.Series(dtype='object')
#     for i in range(len(os.listdir(folder))):
#         dts_timestamps.loc[i] = pd.to_datetime(os.listdir(folder)[i][9:19], format="%H%M%S%f").time()
#     if folder[-2:]=='54':
#         dts_timestamps.to_pickle('dts_exp54_timestamps.pkl')    
#     elif folder[-2:]=='64':
#         dts_timestamps.to_pickle('dts_exp64_timestamps.pkl')
#     # save temperature data
#     all_files  = os.listdir(folder)
#     dts_df = pd.DataFrame(())
#     for i in range(len(all_files)):
#         my_file = os.listdir(folder)[i]
#         file_path = os.path.join(folder, my_file)
#         new_dts   = pd.read_csv(file_path, usecols=[2])
#         dts_df = pd.concat([dts_df, new_dts], ignore_index=True, axis=1)
#     dts_postprocess = dts_df.iloc[-size:]
#     dts_postprocess.to_pickle(save_name)

In [ ]:
# EXPERIMENT 54 ONLY 

### DAS data ###
das_exp54 = pd.read_pickle('data_exp54.pkl')
time_complete_das_exp54 = np.array([loadmat('time_complete_54.mat')['time']], dtype='datetime64[us]').reshape(-1)
trange_54, xrange_54 = mdates.date2num(time_complete_das_exp54), np.arange(4950, 5150)
print('Experiment 54 DAS shape: {}'.format(das_exp54.shape))
print('Times for Experiment 54 shape: {}'.format(time_complete_das_exp54.shape))
print('Timerange 54: {} | X-range 54: {}'.format(trange_54.shape, xrange_54.shape))
print('\n')
### DTS data ###
dts_exp54 = pd.read_pickle('dts_exp54.pkl')
dts_timestamp, dts_depthstamp  = pd.read_pickle('dts_exp54_timestamps.pkl'), pd.read_pickle('dts_depthstamps.pkl')
print('Experiment 54 DTS shape: {}'.format(dts_exp54.shape))
print('Experiment 54: times={} | depths={}'.format(dts_timestamp.shape, dts_depthstamp.shape))

In [ ]:
# fig, ax1 = plt.subplots(1, 1, figsize=(20,10))
# im1 = ax1.imshow(das_exp54, aspect='auto', cmap='seismic', vmin=-50, vmax=50, extent=[trange_54[0], trange_54[-1], xrange_54[-1], xrange_54[0]])
# ax1.xaxis_date(); ax1.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S'))
# ax1.set_xlabel('Time [s] on 06/20/2019'); ax1.set_ylabel('Distance [m]'); ax1.set_title('DAS Experiment 54')
# cbar = plt.colorbar(im1, ax=ax1, pad=0.01)
# cbar.set_label('DAS Amplitude', rotation=270)
# fig.autofmt_xdate()
# plt.show();

# fig, ax2 = plt.subplots(1, 1, figsize=(20,10))
# im2 = ax2.imshow(dts_exp54, aspect='auto', cmap='seismic', vmin=-10, vmax=200, extent=[trange_54[0], trange_54[-1], xrange_54[-1], xrange_54[0]])
# ax2.xaxis_date(); ax2.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S'))
# ax2.set_title('DTS Experiment 54'); ax2.set_xlabel('Time [s] on 06/20/2019'); ax2.set_ylabel('Distance [m]')
# cbar = plt.colorbar(im2, ax=ax2, pad=0.01)
# cbar.set_label('DTS Temperature', rotation=270)
# fig.autofmt_xdate()
# plt.show();

In [ ]:
folder = 'E:/Lytt Fiber Optics/dts 128'
files = os.listdir(folder)
dts = pd.DataFrame()
for i in range(len(files)):
    df = pd.read_csv(os.path.join(folder,files[i])).iloc[:,1]
    dts = pd.concat([dts, df], ignore_index=True, axis=1)
dts_final = dts[-200:]
dts_final.to_pickle('E:/Lytt Fiber Optics/dts_exp128.pkl')
print(dts_final.shape)

In [ ]:
# das_dec = signal.decimate(signal.decimate(das,10),10)

# flowrates = np.zeros((200,4))
# flowrates[115] = exp_flow[0]
# flowrates[126] = exp_flow[1]
# flowrates[138] = exp_flow[2]
# flowrates[142] = exp_flow[3]
# flowrates[152] = exp_flow[4]

# #X_das = np.expand_dims(my_normalize(das_dec.T, mode='minmax'), -1)
# #X_dts = np.expand_dims(my_normalize(dts_exp54.T, mode='minmax'), -1)
# X_das = np.expand_dims(Normalizer('max').fit_transform(das_dec.T), -1)
# X_dts = np.expand_dims(MinMaxScaler().fit_transform(dts.T), -1)
# y_flow = MinMaxScaler().fit_transform(flowrates)
# print('DAS norm: ', X_das.shape)
# print('DTS norm: ', X_dts.shape)
# print('Flow norm:', y_flow.shape)

# X_das_train = X_das[:int(X_das.shape[0]*0.8)]
# X_das_test  = X_das[int(X_das.shape[0]*0.8):]

# X_dts_train = X_dts[:int(X_dts.shape[0]*0.9)]
# X_dts_test  = X_dts[int(X_dts.shape[0]*0.9):]

# print('DAS: Train={} | Test={}'.format(X_das_train.shape, X_das_test.shape))
# print('DTS: Train={}    | Test={}'.format(X_dts_train.shape, X_dts_test.shape))

In [ ]:
# das_train_pred = das_m2m.predict(X_das_train).squeeze().astype('float64')
# das_test_pred = das_m2m.predict(X_das_test).squeeze().astype('float64')
# print('Predictions: train={} | test={}'.format(das_train_pred.shape, das_test_pred.shape))

# das_train_z = das_m2z.predict(X_das_train).squeeze().astype('float64')
# das_test_z = das_m2z.predict(X_das_test).squeeze().astype('float64')
# print('Latent Space: train={} | test={}'.format(das_train_z.shape, das_test_z.shape))

# plt.figure(figsize=(10,4))
# plt.subplot(121)
# plt.imshow(das_train_z.reshape((das_train_z.shape[0], das_train_z.shape[1]*das_train_z.shape[-1])).T, aspect='auto', cmap='binary', vmin=0, vmax=1); 
# plt.colorbar(); plt.show()
# plt.subplot(122)
# plt.imshow(das_test_z.reshape((das_test_z.shape[0], das_test_z.shape[1]*das_test_z.shape[-1])).T, aspect='auto', cmap='binary', vmin=0, vmax=1)
# plt.show()

# plt.figure(figsize=(30,8), facecolor='white')
# plt.subplot(231)
# plt.imshow(X_das_train.squeeze().T, cmap='seismic', aspect='auto', vmin=-0.1, vmax=0.1)
# plt.colorbar(); plt.title('DAS Original - Train')
# plt.subplot(232)
# plt.imshow(das_train_pred.T, cmap='seismic', aspect='auto', vmin=-0.1, vmax=0.1)
# plt.colorbar(); plt.title('DAS Reconstruction - Train')
# plt.subplot(233)
# plt.imshow((X_das_train.squeeze().T - das_train_pred.T)/X_das_train.squeeze().T, cmap='seismic', aspect='auto', vmin=-1, vmax=1)
# plt.colorbar(); plt.title('Normalized Relative Error - Train')
# plt.subplot(234)
# plt.imshow(X_das_test.squeeze().T, cmap='seismic', aspect='auto', vmin=-0.1, vmax=0.1)
# plt.colorbar(); plt.title('DAS Original - Test')
# plt.subplot(235)
# plt.imshow(das_test_pred.T, cmap='seismic', aspect='auto', vmin=-0.1, vmax=0.1)
# plt.colorbar(); plt.title('DAS Reconstruction - Test')
# plt.subplot(236)
# plt.imshow((X_das_test.squeeze().T - das_test_pred.T)/X_das_test.squeeze().T, cmap='seismic', aspect='auto', vmin=-1, vmax=1)
# plt.colorbar(); plt.title('Normalized Relative Error - Test')
# plt.show();

# das_pred = np.vstack((das_train_pred, das_test_pred))
# plt.figure(figsize=(15,4))
# plt.subplot(131)
# plt.imshow(X_das.squeeze().T, cmap='seismic', aspect='auto', vmin=-0.1, vmax=0.1)
# plt.colorbar(); plt.title('DAS Original')
# plt.subplot(132)
# plt.imshow(das_pred.T, cmap='seismic', aspect='auto', vmin=-0.1, vmax=0.1)
# plt.colorbar(); plt.title('DAS Reconstruction')
# plt.subplot(133)
# plt.imshow((X_das.squeeze().T - das_pred.T)/X_das.squeeze().T, cmap='seismic', aspect='auto', vmin=-1, vmax=1)
# plt.colorbar(); plt.title('Normalized Relative Error')
# plt.show()

In [ ]:
# dts_train_pred = dts_m2m.predict(X_dts_train).squeeze().astype('float64')
# dts_test_pred = dts_m2m.predict(X_dts_test).squeeze().astype('float64')
# print('Predictions: train={} | test={}'.format(dts_train_pred.shape, dts_test_pred.shape))
# dts_train_z = dts_m2z.predict(X_dts_train).squeeze().astype('float64')
# dts_test_z = dts_m2z.predict(X_dts_test).squeeze().astype('float64')
# print('Latent Space: train={} | test={}'.format(dts_train_z.shape, dts_test_z.shape))

# plt.subplot(122)
# plt.imshow(dts_test_z.reshape((dts_test_z.shape[0], dts_test_z.shape[1]*dts_test_z.shape[-1])).T, aspect='auto', cmap='binary', vmin=0, vmax=1)

# print('Testing:')
# print('SSIM={:.3f}'.format(structural_similarity(X_dts_test.squeeze().T, dts_test_pred.T)))
# print('MSE={:.3e}'.format(mean_squared_error(X_dts_test.squeeze().T, dts_test_pred.T)))

# plt.figure(figsize=(30,8), facecolor='white')
# plt.subplot(231)
# plt.imshow(X_dts_train.squeeze().T, cmap='seismic', aspect='auto', vmin=0, vmax=1)
# plt.colorbar(); plt.title('DTS Original - Train')
# plt.subplot(232)
# plt.imshow(dts_train_pred.T, cmap='seismic', aspect='auto', vmin=0, vmax=1)
# plt.colorbar(); plt.title('DTS Reconstruction - Train')
# plt.subplot(233)
# plt.imshow((X_dts_train.squeeze().T - dts_train_pred.T)/X_dts_train.squeeze().T, cmap='seismic', aspect='auto')
# plt.colorbar(); plt.title('Normalized Relative Error - Train')
# plt.subplot(234)
# plt.imshow(X_dts_test.squeeze().T, cmap='seismic', aspect='auto', vmin=0, vmax=1)
# plt.colorbar(); plt.title('DTS Original - Test')
# plt.subplot(235)
# plt.imshow(dts_test_pred.T, cmap='seismic', aspect='auto', vmin=0, vmax=1)
# plt.colorbar(); plt.title('DTS Reconstruction - Test')
# plt.subplot(236)
# plt.imshow((X_dts_test.squeeze().T - dts_test_pred.T)/X_dts_test.squeeze().T, cmap='seismic', aspect='auto')
# plt.colorbar(); plt.title('Normalized Relative Error - Test')
# plt.show()

# dts_pred = np.vstack((dts_train_pred, dts_test_pred))
# plt.figure(figsize=(15,4))
# plt.subplot(131)
# plt.imshow(X_dts.squeeze().T, cmap='seismic', aspect='auto', vmin=0, vmax=1)
# plt.colorbar(); plt.title('DTS Original')
# plt.subplot(132)
# plt.imshow(dts_pred.T, cmap='seismic', aspect='auto', vmin=0, vmax=1)
# plt.colorbar(); plt.title('DTS Reconstruction')
# plt.subplot(133)
# plt.imshow((X_dts.squeeze().T - dts_pred.T)/X_dts.squeeze().T, cmap='seismic', aspect='auto')
# plt.colorbar(); plt.title('Normalized Relative Error')

In [ ]:
# 64
exp_flow = np.array([[3000, 100000,   0,    0],
                     [250,  8333.33,  250,  0],
                     [250,  8333.33,  100,  0],
                     [0,    0,        0,    0],
                     [250,  8333.33,  250,  0]])

# 128
exp_flow = np.array([[3000, 60000,  0,   0],
                     [500,  20000,  0,   0],
                     [500,  20000,  0,   0],
                     [0,    0,      0,   0],
                     [500,  20000,  0,   5]])

In [ ]:
das = pd.read_pickle('E:/Lytt Fiber Optics/data_exp128.pkl')
dts = pd.read_pickle('E:/Lytt Fiber Optics/dts_exp128.pkl')
exp_flow = np.array([[3000, 60000,  0,   0],
                     [500,  20000,  0,   0],
                     [500,  20000,  0,   0],
                     [0,    0,      0,   0],
                     [500,  20000,  0,   5]])

das_dec = signal.decimate(signal.decimate(das,10),10)

flowrates = np.zeros((200,4))
flowrates[115] = exp_flow[0]
flowrates[126] = exp_flow[1]
flowrates[138] = exp_flow[2]
flowrates[142] = exp_flow[3]
flowrates[152] = exp_flow[4]

X_das = np.expand_dims(Normalizer('max').fit_transform(das_dec.T), -1)
X_dts = np.expand_dims(MinMaxScaler().fit_transform(dts.T), -1)
y_flow = MinMaxScaler().fit_transform(flowrates)

das_z = das_m2z.predict(X_das).squeeze().astype('float64')
dts_z = dts_m2z.predict(X_dts).squeeze().astype('float64')

z_dual = np.concatenate([das_z, dts_z])
y_flow_exp = np.repeat(y_flow[np.newaxis, :, :], z_dual.shape[0], axis=0)

y_flow_pred = flow_net.predict(z_dual)
mse  = mean_squared_error(y_flow_exp, y_flow_pred)
ssim = structural_similarity(y_flow_exp, y_flow_pred, channel_axis=-1)
print('MSE={:.3e} | SSIM={:.5f}'.format(mse,ssim))

In [ ]:
plt.figure(figsize=(8,10), facecolor='white')

plt.subplot(211)
plt.imshow(y_flow_exp[0], aspect='auto', cmap='turbo')
plt.title('Normalized Injection Rate Map'); plt.colorbar()
plt.xticks([0,1,2,3], labels=['oil','water','gas','sand']); plt.ylabel('distance')

plt.subplot(212)
plt.imshow(y_flow_pred.mean(0), aspect='auto', cmap='turbo', vmin=0, vmax=1)
plt.title('Predicted Injection Rate Map'); plt.colorbar()
plt.xticks([0,1,2,3], labels=['oil','water','gas','sand']); plt.ylabel('distance')

In [ ]:
np.save('models/z_train_dual.npy', z_train_dual)
np.save('models/z_test_dual.npy', z_test_dual)

PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(svd_solver='full', whiten=True)
pca.fit(z_train_dual.reshape(z_train_dual.shape[0], z_train_dual.shape[1]*z_train_dual.shape[-1]))
z_pca = pca.transform(z_train_dual.reshape(z_train_dual.shape[0], z_train_dual.shape[1]*z_train_dual.shape[-1]))

plt.figure()
plt.hist(pca.components_, histtype='stepfilled', bins=20, density=True, edgecolor='k')
plt.vlines(pca.components_.mean() + 2*pca.components_.std(), 0, 15, 'k', linestyle='--', linewidth=2)
plt.vlines(pca.components_.mean() - 2*pca.components_.std(), 0, 15, 'k', linestyle='--', linewidth=2)
plt.xlim([-.20,.20]); plt.legend(range(10))
plt.show()

pca.components_[pca.components_ >= pca.components_.mean() + 2*pca.components_.std()] = 0
pca.components_[pca.components_ <= pca.components_.mean() - 2*pca.components_.std()] = 0

plt.figure()
plt.hist(pca.components_, histtype='stepfilled', bins=20, density=True, edgecolor='k')
plt.xlim([-.2,.2]); plt.legend(range(10))
plt.show()

plt.imshow(z_train_dual.reshape(z_train_dual.shape[0], z_train_dual.shape[1]*z_train_dual.shape[-1]).T, aspect='auto', cmap='magma', vmin=0, vmax=1)
plt.imshow(pca.inverse_transform(z_pca).T, aspect='auto', cmap='magma', vmin=0, vmax=1)